In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pycaret as pc
from tqdm.notebook import tqdm

In [2]:
## Some cool settings
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#93D30C", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
tqdm.pandas()

In [71]:
#read the metadata of the file using pandas
import os,sys,csv
csv.field_size_limit(sys.maxsize)

# Dictionary to store the metadata of the problems
problems_metadata = {}
#Reading the folder names
folder_names = os.listdir("/Users/adirserruya/Desktop/PyCarret tutorial/Multivariate_datasets")
#Only for mac problem with hidden files (DS Store files)
folder_names = [x for x in folder_names if x[0] != "."]
#Iterating through the files to extract the metadata
for folder in folder_names:
    header_start_index = 0
    metadata = pd.read_csv("/Users/adirserruya/Desktop/PyCarret tutorial/Multivariate_datasets/"+folder+"/"+folder+"_TRAIN.ts",
                       engine ="python",header=None, nrows=200 ,dtype=str,delimiter="#")
    metadata[0] = metadata[0].astype(str)
    meta_dict = {}
    # Reading the actual metaData from the file
    for row in metadata[0][:-1]:
        if row[0] == "@":
            splity = row.split(" ")
            meta_dict[splity[0].lstrip("@")] = splity[1:]
    # Saving the relevant metadata in a dictionary
    
    #create list of alphabetical characters
    alphabet = [chr(i) for i in range(97, 123)]
    
    
    
    for row in metadata[0][:-1]:
        if row[0] == "@" or row[0] in alphabet:
            header_start_index += 1
    try:
        PROBLEM_NAME = meta_dict["problemName"][0]
        TIME_STAMPS = bool(meta_dict["timeStamps"][0])
        MISSING_VALUES = bool(meta_dict["missing"][0])
        UNIVARIATE = bool(meta_dict["univariate"][0])
        NUM_FEATURES = int(meta_dict["dimensions"][0])
        EQUAL_LENGTH_INPUTS = bool(meta_dict["equalLength"][0])
        SERIES_LENGTH = int(meta_dict["seriesLength"][0])
        POSSIBLE_LABELS = meta_dict["classLabel"][1:]
        problems_metadata[PROBLEM_NAME] = {"TIME_STAMPS":TIME_STAMPS,
                                           "MISSING_VALUES":MISSING_VALUES,
                                            "UNIVARIATE":UNIVARIATE,
                                            "NUM_FEATURES":NUM_FEATURES,
                                            "EQUAL_LENGTH_INPUTS":EQUAL_LENGTH_INPUTS,
                                            "SERIES_LENGTH":SERIES_LENGTH,
                                            "POSSIBLE_LABELS":POSSIBLE_LABELS,
                                            "HEADER_START_INDEX":header_start_index
                                                                }
    except KeyError as e:
        print(f"{folder} : Error in the metadata {e}")
        
    

InsectWingbeat : Error in the metadata 'seriesLength'
CharacterTrajectories : Error in the metadata 'seriesLength'
JapaneseVowels : Error in the metadata 'seriesLength'
SpokenArabicDigits : Error in the metadata 'seriesLength'


In [66]:
metadata[0][:-1]

0                                                    nan
1                                                    nan
2                                                    nan
3                                                    nan
4                                                    nan
                             ...                        
128    -0.80666,-0.80666,-0.80666,-0.80666,-0.80666,-...
129    -0.85196,-0.85196,-0.85196,-0.85196,-0.85196,-...
130    -0.173,-0.173,-0.173,-0.173,-0.173,-0.173,-0.1...
131    1.1111,1.1111,1.1111,1.1111,1.1111,1.1111,1.11...
132    -0.98835,-0.98835,-0.98835,-0.98835,-0.98835,-...
Name: 0, Length: 133, dtype: object

In [19]:
path = "/Users/adirserruya/Desktop/PyCarret tutorial/Multivariate_datasets/"

In [72]:
problems_metadata["BasicMotions"]["HEADER_START_INDEX"]

13

In [78]:
#SpkenArabicDigits

bm_df = pd.read_csv(path+"BasicMotions/BasicMotions_TRAIN.ts",header=problems_metadata["BasicMotions"]["HEADER_START_INDEX"])

In [79]:
problems_metadata["BasicMotions"]

{'TIME_STAMPS': True,
 'MISSING_VALUES': True,
 'UNIVARIATE': True,
 'NUM_FEATURES': 6,
 'EQUAL_LENGTH_INPUTS': True,
 'SERIES_LENGTH': 100,
 'POSSIBLE_LABELS': ['Standing', 'Running', 'Walking', 'Badminton'],
 'HEADER_START_INDEX': 13}

In [80]:
bm_df

,0.079106,0.079106.1,-0.903497,1.116125,1.6382,1.003448,0.028774,0.03005,-0.120485,-0.120485.1,...,-0.039951.2,-0.002663.5,0.047941.3,0.071911,0.063921.5,-0.021307.3,-0.042614.2,-0.063921.1,-0.063921.2,-0.03196:Standing
0,0.377751,0.377751,2.952965,4.310925,3.256906,0.850679,-0.909276,-0.909276,0.205036,-0.480176,...,0.058594,0.143822,0.122515,0.039951,-0.039951,-0.039951,-0.135832,-0.135832,0.034624,0.034624:Standing
1,-0.813905,-0.813905,-0.424628,0.316895,0.228580,-0.162312,-0.162312,0.002101,0.115463,-0.213192,...,0.013317,-0.215733,-0.258347,-0.106535,-0.007990,-0.077238,-0.167792,-0.372872,-0.423476,-0.423476:Standing
2,0.289855,0.289855,-0.669185,0.032063,-0.214649,0.161245,0.161245,0.374508,0.009226,-0.221630,...,0.063921,-0.335585,-0.218397,-0.143822,-0.143822,0.207743,0.436793,0.420813,0.375536,0.074574:Standing
3,-0.123238,-0.123238,-0.249547,-0.272929,-0.272929,-0.247999,-0.247999,0.057938,-0.414336,-0.209641,...,0.063921,0.122515,0.125179,0.125179,0.047941,0.047941,-0.117188,-0.141159,-0.141159,0.0:Standing
4,-0.357300,-0.357300,-0.005055,0.188406,-0.005550,-0.375081,-0.283544,0.211903,0.025877,-0.362505,...,0.058594,0.042614,0.045277,0.013317,0.002663,0.018644,0.021307,0.010653,0.053267,0.042614:Standing
5,1.236069,1.236069,0.118106,-0.311451,0.800338,0.844668,0.131246,-0.480234,0.260514,0.260514,...,-0.013317,0.007990,0.039951,0.074574,0.069248,0.103872,0.058594,-0.042614,-0.135832,-0.295634:Standing
6,-0.366403,-0.366403,0.126830,0.126830,-0.284847,0.301496,0.218163,-0.292671,-0.177331,-0.372362,...,0.103872,0.103872,0.567298,0.567298,0.167792,-0.125179,-0.125179,-0.503377,-0.503377,-0.476744:Standing
7,-0.342233,-0.342233,-0.298542,-0.326765,-0.259437,0.437525,0.097106,-0.473853,0.002809,-0.049060,...,0.303624,0.471417,0.364882,0.018644,-0.237040,-0.538001,-0.519358,-0.287644,0.234377,0.476744:Standing
8,-0.407421,-0.407421,2.355158,1.147434,0.142240,-0.181678,-0.833735,0.129367,0.922358,0.797723,...,0.000000,0.090555,-0.087891,-0.114525,-0.130505,-0.007990,0.106535,0.143822,0.138495,0.085228:Standing
9,0.300413,0.300413,-1.964993,-3.159662,-6.498703,0.898168,12.391905,12.721008,18.877642,15.903170,...,-1.371637,1.145250,4.599644,5.097695,3.379820,0.759061,-4.445169,-4.778091,-2.897749,-0.234377:Running


In [86]:
bm_df.shape

(39, 595)

In [119]:
labels = []
count = 0
count_2 = 0
samples = []
for row in range(bm_df.shape[0]):
    for sample in bm_df.iloc[row]:
        count += 1
        if type(sample) == str:
            count_2 += 1
            if sample.split(":")[1] in problems_metadata["BasicMotions"]["POSSIBLE_LABELS"]:
                labels.append(sample.split(":")[1])
                samples.append(sample.split(":")[0])
            else:
                samples.append(sample.split(":")[0])
                samples.append(sample.split(":")[1])
            print(f"Series number {count_2} was found after {count}")
        else:
            samples.append(sample)

        

        
x_train = pd.DataFrame(np.array(samples).reshape(-1, 6))
y_train = pd.DataFrame(labels)



Series number 1 was found after 100
Series number 2 was found after 199
Series number 3 was found after 298
Series number 4 was found after 397
Series number 5 was found after 496
Series number 6 was found after 595
Series number 7 was found after 695
Series number 8 was found after 794
Series number 9 was found after 893
Series number 10 was found after 992
Series number 11 was found after 1091
Series number 12 was found after 1190
Series number 13 was found after 1290
Series number 14 was found after 1389
Series number 15 was found after 1488
Series number 16 was found after 1587
Series number 17 was found after 1686
Series number 18 was found after 1785
Series number 19 was found after 1885
Series number 20 was found after 1984
Series number 21 was found after 2083
Series number 22 was found after 2182
Series number 23 was found after 2281
Series number 24 was found after 2380
Series number 25 was found after 2480
Series number 26 was found after 2579
Series number 27 was found afte

In [255]:
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

## Creating a custom dataset class for timeseries
class TimeSeriesDataset(Dataset):
    def __init__(self, x, y,seq_length,transform=None):
        labelEncoder = LabelEncoder()
        self.y = labelEncoder.fit_transform(y).astype("float32")
        self.x = x.astype("float32")
        self.seq_length = seq_length
        self.transform = transform
        self.idy = 0
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx ):
        idy = self.idy
        data,target = self.x[idx:idx+self.seq_length].to_numpy(), self.y[idy]
        idx += self.seq_length
        self.idy += 1
        return torch.Tensor(data).squeeze(0),torch.Tensor([target])

In [256]:
#Creating the dataloader
train_loader = DataLoader(TimeSeriesDataset(x_train,y_train,problems_metadata["BasicMotions"]["SERIES_LENGTH"]),batch_size=10,shuffle=False)


In [257]:
train,target = next(iter(train_loader))

In [258]:
train.shape

torch.Size([10, 100, 6])

In [259]:
target.shape

torch.Size([10, 1])